# Model 000 basic

### Charlie working on

This model has stream power and linear diffusion of the forms:
$$\frac{d\eta}{dt}=-KA^mS^n$$
and
$$\frac{d\eta}{dt} = -D\nabla^2\eta$$

So, at steady state, the landscape will obey:
$$S=\left(\frac{U}{KA^m}\right)^{1/n}$$
when only the fluvial rule is in place ($D=0$) and:
$$\eta = -\frac{U}{D}\frac{x^2}{2}+\frac{U}{D}\frac{L^2}{2}$$
when only the hillslope rules are in place ($K=0$).

In [1]:
from terrainbento import Basic
import numpy as np
import matplotlib.pyplot as plt
#from landlab import RasterModelGrid
#from landlab.components import DepressionFinderAndRouter
#from landlab.components.flow_routing import FlowRouter
#from landlab.components import FastscapeEroder
#from landlab.components import LinearDiffuser
from landlab import imshow_grid

### There will be two parts: 1) turn off channels and just do hillslopes, and 2) turn off hillslopes and just do channels. Both will start as flat+noise

### Part 1: turn off channels and just do hillslopes

In [2]:
#parameter dictionary
params = { 'number_of_node_rows' : 10,
          'number_of_node_columns' : 100,
          'node_spacing' : 10.0,
          'east_boundary_closed' : False,
          'north_boundary_closed' : True,
          'west_boundary_closed' : False,
          'south_boundary_closed' : True,
          'dt' : 10.0,
          'K_sp' : 0,
          'm_sp' : 0.5,
          'n_sp' : 1.0,
          'linear_diffusivity' : 1.0,
          'outlet_lowering_rate' : 0.0005,
          'output_filename': 'model_000_output'
}

In [3]:
basic = Basic(params=params)
#channel_erodibility = 0 #m/yr
#hillslope_diffusivity = 1. #m^2/yr
#uplift_rate = 0.001 #m/yr
tolerance = 0.0001

#run_time = 100000 #years
#dx = 1.0
#dt = 10. #year
#nr = 10
#nc = 100
#mg = RasterModelGrid((nr, nc), dx)
#np.random.seed(seed = 5000)
#mg.add_zeros('node', 'topographic__elevation')

#mg['node']['topographic__elevation'] += np.random.rand(len(mg.node_y)) / 10000
#ld = LinearDiffuser(mg, linear_diffusivity = hillslope_diffusivity)

#%matplotlib inline
#imshow_grid(mg, 'topographic__elevation')

AssertionError: 

In [ ]:
#basic.grid.set_closed_boundaries_at_grid_edges(bottom_is_closed=True,left_is_closed=False,right_is_closed=False,top_is_closed=True)
dt = 10
#basic.run_for(1, 100000)
#K_ss = None
#print(K_ss)
elapsed_time = 0 #years
keep_running = True
while keep_running == True:
    pre_topo = basic.grid.at_node['topographic__elevation'][basic.grid.core_nodes]
    basic.run_one_step(dt)
    #    ld.run_one_step(dt = dt)
    #basic.grid.at_node['topographic__elevation'][basic.grid.boundary_nodes] -= uplift_rate * dt
    post_topo = basic.grid.at_node['topographic__elevation'][basic.grid.core_nodes]
    if elapsed_time % 1000 == 0:
        print(elapsed_time)
        print(max(abs(pre_topo - post_topo)))
    elapsed_time += dt
    if max(abs(pre_topo - post_topo)) <= tolerance: #1mm
        keep_running = False

In [ ]:
basic.grid.open_boundary_nodes
max(abs(pre_topo - post_topo))
#%matplotlib inline
#imshow_grid(basic.grid, 'topographic__elevation')

In [ ]:
%matplotlib inline
imshow_grid(basic.grid, 'topographic__elevation')

In [ ]:
import matplotlib
from matplotlib import ticker
#
topo = basic.grid.at_node['topographic__elevation']


#plotting param
matplotlib.rcParams.update({'font.size': 16})

# #instantiate figure and plot
topo_fig = plt.figure(figsize=(6, 1))
t1 = plt.subplot()
topo = topo.reshape(10,100)
ts1_plot = t1.imshow(topo[::-1], cmap='terrain', vmin = 0, vmax = 60)

#add colorbar
cb = plt.colorbar(ts1_plot, label = 'Elevation [m]')
tick_locator = ticker.MaxNLocator(nbins=3)
cb.locator = tick_locator
cb.update_ticks()

#axis labels
t1.tick_params(labelbottom='off', labelleft='off') 
t1.set_ylabel('100 m', labelpad = 15)
t1.set_xlabel('1 km side length', labelpad = 15)

#save figure
topo_fig.savefig('linear_diffusion_topo.eps',bbox_inches='tight', dpi=300)


In [ ]:
basic.grid.open_boundary_nodes
basic.outlet_node
print(basic.grid.at_node['topographic__elevation'][1])

In [ ]:
uplift_rate = 0.0005
hillslope_diffusivity = 1.0
dx = 10.0
topo_profile = basic.grid.at_node['topographic__elevation'][basic.grid.node_y == 50]
domain = np.arange(0, max(basic.grid.node_x + dx), dx)
plt.plot(domain, topo_profile)
x = np.arange(-max(domain) / 2., max(domain) / 2. + dx, dx)
theory_profile = (uplift_rate / hillslope_diffusivity) * ((max(x+max(x))/2.)**2 / 2.0) + ((-uplift_rate / hillslope_diffusivity) * (x**2 / 2.0))
plt.plot(x + max(x), theory_profile , linestyle='--')
#print(topo_profile)
#print(theory_profile)
#print(x)
#print(domain)
#print(max(x+max(x)))
#print(uplift_rate)

In [ ]:
import matplotlib
#define "domain" and "topo"
topo = basic.grid.at_node['topographic__elevation'][basic.grid.node_y == 50]
domain = np.arange(0, max(basic.grid.node_x + dx), dx)

##instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
hillslope = plt.subplot()

#plotting param
matplotlib.rcParams.update({'font.size': 20})

#parameters
uplift_rate = 0.0005
hillslope_diffusivity = 1.0

#plot the actual profile
hillslope.plot(domain / 1000, topo, marker='o', c='k', linewidth = 0, markerfacecolor='None', label = 'Numerical Solution')

#plot the theoretical profile
x = np.arange(-max(domain) / 2., max(domain) / 2. + dx, dx)
theory_profile = (uplift_rate / hillslope_diffusivity) * \
    ((max(x+max(x))/2.)**2 / 2.0) + ((-uplift_rate / hillslope_diffusivity) *\
    (x**2 / 2.0))
plt.plot((x + max(x)) / 1000, theory_profile , linestyle='-', c='grey', linewidth = 2, label = 'Analytical Solution')

#axis labels
hillslope.set_xlabel('Distance [km]')
hillslope.set_ylabel('Elevation [m]')

#legend
hillslope.legend(scatterpoints = 1, prop={'size':12})

#save figure
fig.savefig('basic_lin_diff_topo.eps',bbox_inches='tight', dpi=300)

Linear hillslope diffusion: check!!

### Part 2: turn off hillslopes and just do channels

In [ ]:
#parameter dictionary (diffusivity will be 0, K_sp is nonzero)
params = { 'number_of_node_rows' : 100,
          'number_of_node_columns' : 160,
          'node_spacing' : 10.0,
          'east_boundary_closed' : False,
          'north_boundary_closed' : False,
          'west_boundary_closed' : False,
          'south_boundary_closed' : False,
          'dt' : 10.0,
          'K_sp' : 0.001,
          'm_sp' : 0.5,
          'n_sp' : 1.0,
          'linear_diffusivity' : 10e-20,
          'outlet_lowering_rate' : 0.0005,
          'output_filename': 'model_000_output'
}

In [ ]:
#instantiate and establish topo error tolerance
basic = Basic(params=params)
tolerance = 0.0001

In [ ]:
#time loop
dt = 10
elapsed_time = 0 #years
keep_running = True
while keep_running == True:
    pre_topo = basic.grid.at_node['topographic__elevation'][basic.grid.core_nodes]
    basic.run_one_step(dt)
    #    ld.run_one_step(dt = dt)
    #basic.grid.at_node['topographic__elevation'][basic.grid.boundary_nodes] -= uplift_rate * dt
    post_topo = basic.grid.at_node['topographic__elevation'][basic.grid.core_nodes]
    if elapsed_time % 1000 == 0:
        print(elapsed_time)
        print(max(abs(pre_topo - post_topo)))
    elapsed_time += dt
    if max(abs(pre_topo - post_topo)) <= tolerance: #1mm
        keep_running = False

In [ ]:
%matplotlib inline
imshow_grid(basic.grid, 'topographic__elevation')

In [ ]:
print basic.grid['node']['topographic__elevation'][300:450]
print keep_running
#basic.grid.open_boundary_nodes

In [ ]:
print basic.grid.at_node['topographic__steepest_slope'][235]
#for some reason, the line of nodes adjacent to all the boundaries
plt.scatter(basic.grid.at_node['drainage_area'][(basic.grid.node_x > 1)&(basic.grid.node_x < 158)&(basic.grid.node_y >1)&(basic.grid.node_y<98)], basic.grid.at_node['topographic__steepest_slope'][(basic.grid.node_x > 1)&(basic.grid.node_x < 158)&(basic.grid.node_y >1)&(basic.grid.node_y<98)])
plt.plot(basic.grid.at_node['drainage_area'][(basic.grid.node_x > 1)&(basic.grid.node_x < 158)&(basic.grid.node_y >1)&(basic.grid.node_y<98)], (0.0005 / 0.01)*np.power(basic.grid.at_node['drainage_area'][(basic.grid.node_x > 1)&(basic.grid.node_x < 158)&(basic.grid.node_y >1)&(basic.grid.node_y<98)], -0.5))
plt.xscale('log')
plt.yscale('log')
plt.ylim(0.0003, 0.01)

In [ ]:
###MAKE SLOPE-AREA PLOT

import matplotlib

#assign area_array and slope_array
area_array = basic.grid.at_node['drainage_area'][(basic.grid.node_x > 10)&(basic.grid.node_x < 1580)&(basic.grid.node_y >10)&(basic.grid.node_y<980)]
slope_array = basic.grid.at_node['topographic__steepest_slope'][(basic.grid.node_x > 10)&(basic.grid.node_x < 1580)&(basic.grid.node_y >10)&(basic.grid.node_y<980)]

##instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
slope_area = plt.subplot()

#plotting param
matplotlib.rcParams.update({'font.size': 20})

#create an array for the detachment-limited analytical solution
u = 0.0005 #m/yr, uplift or baselevel lowering rate
k = 0.001 #fluvial erodibility
m = 0.5 #discharge exponent
n = 1.0 #slope exponent

#calculate analytical slope from area field
analytical_slope_array = np.power((u / k), 1 / n) * np.power(area_array, -m/n)

#plot the analytical solution
slope_area.plot(area_array, analytical_slope_array, linestyle='-',
                color='grey', linewidth = 1, label = 'Analytical solution')

#plot the data
slope_area.scatter(area_array, slope_array, marker='o', c='k', 
                   label = 'Numerical solution') #plot HA data
                   
#make axes log and set limits
slope_area.set_xscale('log')
slope_area.set_yscale('log')

slope_area.set_xlim(9*10**1, 2*10**5)
slope_area.set_ylim(1e-3, 1e-1)

#set x and y labels
slope_area.set_xlabel(r'Drainage area [m$^2$]')
slope_area.set_ylabel('Channel slope [-]')
slope_area.legend(scatterpoints=1,prop={'size':12})
slope_area.tick_params(axis='x', which='major', pad=7)

fig.savefig('basic_streampower_slope_area.eps',bbox_inches='tight', dpi=1000) #save figure

In [ ]:
##MAKE TOPO FIGURE
#define 'topo'
topo = basic.grid.at_node['topographic__elevation']
#plotting param
matplotlib.rcParams.update({'font.size': 20})

# #instantiate figure and plot
topo_fig = plt.figure(figsize=(6, 3.75))
t1 = plt.subplot()
topo = topo.reshape(100,160)
ts1_plot = t1.imshow(topo[::-1], cmap='terrain', vmin = 0, vmax = 3.5)

#add colorbar
cb = plt.colorbar(ts1_plot, label = 'Elevation [m]')

#axis labels
t1.tick_params(labelbottom='off', labelleft='off') 
t1.set_ylabel('1 km side length', labelpad = 15)
t1.set_xlabel('1.6 km side length', labelpad = 15)

#save figure
topo_fig.savefig('basic_streampower_topo.eps',bbox_inches='tight', dpi=300)

In [ ]:
from landlab.io.netcdf import write_netcdf
write_netcdf('basic_stream_power.nc', basic.grid, format='NETCDF3_64BIT', names='topographic__elevation')

In [ ]:
area_array